In [1]:
import re
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt


In [2]:
seqfile = '/global/scratch/projects/fc_mvslab/data/sequencing/CZB_Feb2024/A10_A11/results/A11_2_sequences.txt'
testfile = '/global/scratch/projects/fc_mvslab/data/sequencing/CZB_Feb2024/A10_A11/results/A11_2_sequences_less.txt'
a10design = 'a11_designfile.csv'

In [3]:
def complement(seqn):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N', 'X':'X'} 
    bases = list(seqn) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
def reverse_complement(s):
        return complement(s[::-1])

def design_dict(des):
    dt = []
    with open(des, 'r') as f_des:
        # open the design file for reading
        for line in f_des:
            if "ArrayDNA" in line:
                pass
            else:
                dt.append(line.strip())
    
    dt_dict = {}
    for i in dt:
        dt_dict[i] = 1
        
    print(len(dt_dict))
    return(dt_dict)
        
design_dict(a10design)

18788


{'ATGGTTGAAAATTCTACTCAGAAAGCTCCACATGCTGGTAATGATGATAATTCTTCTACTAAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCT': 1,
 'ACTCAGAAAGCTCCACATGCTGGTAATGATGATAATTCTTCTACTAAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACT': 1,
 'CATGCTGGTAATGATGATAATTCTTCTACTAAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAA': 1,
 'GATAATTCTTCTACTAAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAAAATTCTGAAACTCAT': 1,
 'AAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAAAATTCTGAAACTCATAATAGGAAGAGAAAT': 1,
 'GCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAAAATTCTGAAACTCATAATAGGAAGAGAAATAGAATCTTGTTTGTT': 1,
 'TTTAATAATCCAACTCCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAAAATTCTGAAACTCATAATAGGAAGAGAAATAGAATCTTGTTTGTTTGTCAAGCTTGTAGA': 1,
 'CCAGGTTTGGAAGCTGAACATTCTTCTACTTCTCCAGCTCCAGAAAATTCTGAAACTCATAATAGGAAGAGAAATAGAATCTTGTTTGTTTGTCAAGCTTGT

In [4]:
dtiles = design_dict(a10design)

18788


In [5]:
#always check your dictionary works
dtiles['ATGGTTGAAAATTCTACTCAGAAAGCTCCACATGCTGGTAATGATGATAATTCTTCTACTAAACCATATTCTGAAGCATTCTTCTTGGGATTTAATAATCCAACTCCAGGTTTGGAAGCT']

1

In [6]:
def tilebc_mapper(read, design_dict,
                  ad_preceder = "GGCTAGC", ad_length = 120, 
                  bc1_preceder = "CGCGCC", bc1length = 11,
                  bc2_preceder = "CTCGAG", bc2length = 14):
    """
    read: 
    ad_preceder:
    ad_length:
    bc1_preceder:
    bc1_length:
    bc2_preceder:
    bc2_length:
    """
    
    #Initial ad search
    searched_read = re.split(ad_preceder, read, maxsplit=1) 
    AD = None
    barcode1 = None
    barcode2 = None
    designed = 0
    
    if len(searched_read) == 2: #if the first ad search was successful
        roi = searched_read[1] #then take the ad search
#         print("roi =", roi)
        AD = roi[:ad_length] #and save the AD as the first part of the string
        
        #Check if AD Tile was designed
        if design_dict:
            if AD in design_dict:
#                 print('yes designed')
                designed = 1
#             else:
#                 print('not designed')

        
        #Initial bc1 search
        searched_read = re.split(bc1_preceder, roi[ad_length:], maxsplit=1) #look for the barcode using the bc preceder in the portion of the read beyond the AD
        if len(searched_read) == 2: #if initial bc1 search was successful
            barcode1 = searched_read[1][:bc1length] #then the barcode is the first 11 characters of the search result
        else: #if bc search via preceder was unsuccessful
            #Secondary bc1 search
            barcode1 = roi[139:150]
    #         print(len(barcode))

        #Initial bc2 search
        searched_read = re.split(bc2_preceder, roi[ad_length:], maxsplit=1) #look for the barcode using the bc preceder in the portion of the read beyond the AD
    #     print(searched_read)
        if len(searched_read) == 2: #if search was successful
            barcode2 = searched_read[1][:bc2length] #bc2 is the first character of search result
        else: #if bc search via preceder was unsuccessful
            #Secondary bc2 search
            barcode2 = roi[168:182]
    
    return AD, barcode1, barcode2, designed

In [7]:
ADeg = reverse_complement('CAtaactaattacatgaGCGGCCNNNNNNNNNNNNNNXTCGAGCTTAAGGGGCCCNNNNNNNNNNNXGCGCGCCTCAGCTAGTCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNgctagccgatgaagtagagcccgcagt'.upper())
tilebc_mapper(ADeg, dtiles)

('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
 'NNNNNNNNNNN',
 'NNNNNNNNNNNNNN',
 0)

In [8]:
def buildmapdf (readfile, readcount):
    """Takes file of sequences only (readfile) and #lines in that file (readcount)"""
    
    #set up progress updates
    iteration_count = 0 #start a counter for the loop to be able to print progress at N iterations
    total_iterations = readcount 
    progress_counter = readcount/10
    
    total_designed = 0 #to immediately count how many tiles were designed
    start1 = time.perf_counter() #start counter
    
    data = []
    
    with open(readfile, 'r') as fin:
        print("Opened file")

        for line in fin:
            """
            Each iteration is 
            """
            iteration_count += 1
            #clean the read
            seq = line.strip()
            seq = reverse_complement(seq)
            #find ad tile and bcs
            to_df = tilebc_mapper(seq, dtiles) #uses gcn4 designed tiles to check for designed
            #count the number of designed
            total_designed += to_df[3]

            if iteration_count % progress_counter ==0: # progress update
                print((f"Iteration {iteration_count}: Progress {iteration_count/total_iterations*100:.2f}%"))
                progtime = time.perf_counter()
                print(f'iteration time progress: {round(progtime-start1, 2)} seconds')

#             print("results = ", ad, adbc, rptrbc, dquery)
            data.append(to_df)
        
    finish = time.perf_counter()
    print(f'finished processing in {round(finish-start1, 2)} seconds')
    print(f'Total designed = {total_designed} out of {total_iterations}')
#     print(data)
    output_df = pd.DataFrame(data, columns=['AD', 'AD_BC', 'RPTR_BC', 'Designed'])
    finish = time.perf_counter()
    print(f'finished creating df in {round(finish-start1, 2)} seconds')
    return output_df

In [9]:
testmap = buildmapdf(testfile, 10)

Opened file
Iteration 1: Progress 10.00%
iteration time progress: 0.0 seconds
Iteration 2: Progress 20.00%
iteration time progress: 0.0 seconds
Iteration 3: Progress 30.00%
iteration time progress: 0.0 seconds
Iteration 4: Progress 40.00%
iteration time progress: 0.0 seconds
Iteration 5: Progress 50.00%
iteration time progress: 0.0 seconds
Iteration 6: Progress 60.00%
iteration time progress: 0.0 seconds
Iteration 7: Progress 70.00%
iteration time progress: 0.0 seconds
Iteration 8: Progress 80.00%
iteration time progress: 0.0 seconds
Iteration 9: Progress 90.00%
iteration time progress: 0.0 seconds
Iteration 10: Progress 100.00%
iteration time progress: 0.0 seconds
finished processing in 0.0 seconds
Total designed = 1 out of 10
finished creating df in 0.01 seconds


In [10]:
bigmap = buildmapdf(seqfile, 233146565)

Opened file
Iteration 46629313: Progress 20.00%
iteration time progress: 782.92 seconds
Iteration 93258626: Progress 40.00%
iteration time progress: 1566.11 seconds
Iteration 139887939: Progress 60.00%
iteration time progress: 2379.07 seconds
Iteration 186517252: Progress 80.00%
iteration time progress: 3156.57 seconds
Iteration 233146565: Progress 100.00%
iteration time progress: 3945.92 seconds
finished processing in 3946.01 seconds
Total designed = 80914264 out of 233146565
finished creating df in 4211.98 seconds


In [11]:
bigmap.head()

,AD,AD_BC,RPTR_BC,Designed
0,None,None,None,0
1,TTGTTGGATTCTCCACATGATACTTCTCCAGTTAAAGAAACTTTTG...,ACTATGTTGGG,CAAATACTCTTTTC,0
2,TGTCCACAAGGTGGTGAATATTCTGATTTGTTGAGAAATTTGATTA...,GATCAACTAAC,TGGCCTCGGAAGAA,0
3,CCACCACCATTGATCTTGAGAGGTCATGAAGCATTTATTTTTTTAG...,TAATGGTCCCC,ACACTGTAAGTATT,0
4,GATAAATTGCCACATGAAGTTGCTGAACAATTGAGATTGTTGTCTT...,GGAAAGATCTA,CTTGCTGGAACAGC,1


In [13]:
# bigmap.to_csv('CURRENT_A11_2_tbcRAW_v2.csv')

In [14]:
bigmap = bigmap.dropna()
print(bigmap.shape[0])
bigmap = bigmap[bigmap['Designed']==1]
print(bigmap.shape[0])

151966240
80914264


In [16]:
print('% of paired after dropping nan')
print(151966240/233146565)
print('% of paired after dropping non-designed')
print(80914264/233146565)

% of paired after dropping nan
0.6518056142066687
% of paired after dropping non-designed
0.3470532109276412
